# Последняя версия LSTM сети, обучающаяся на большом наборе данных
## Есть train и test ноутбуки

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os

from IPython.display import display
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [2]:
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(400)

In [3]:
#///////////////////////////////// для работы с датасетом

    # путь к файлу, из которого берутся данные для обучения
path_to_positive_data = 'positive.csv' 
path_to_negative_data = 'negative.csv'

    # куда сохранять данные после обработки (вытащит один столбец, убрал англ. символы)
path_to_processed_positive_data = "my_pos_text.csv" 
path_to_processed_negative_data = "my_neg_text.csv"

    # название взятой величины из файла
target = 'text' 

#///////////////////////////////// для колбэков

    # для Early_stopping
ES_patience = 20 # кол-во эпох без улучшений
ES_min_delta = 0.001 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'val_loss' # отслеживаемый параметр 
ES_save_best_weights = True # сохранять ли веса нейронки с лучшими результатами
    
    # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'loss'  # отслеживаемый параметр 
RLPOP_factor = 0.1 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 15 # кол-во эпох без улучшений
RLPOP_verbose = 1 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.0001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 0 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости

    # для CallbackList
CBL_add_history = True # вызывать ли колбэк History (если он не был довавлен вручную)
CBL_add_progbar = True # вызывать ли колбэк ProgbarLogger (если он не был довавлен вручную)

    #///////////////////////////////// для работы со словами
    
maxWordsCount = 6000 # макс слов в словаре
max_text_len = 10 # макс длинна высказывания

    #///////////////////////////////// для компиляции 
    
CMP_learning_rate = 0.2 # шаг сходимости back propogation
CMP_solver = tf.keras.optimizers.SGD(learning_rate = CMP_learning_rate, momentum = 0.8, nesterov = True) # оптимизатор
CMP_loss_func = 'binary_crossentropy' # функция потерь
CMP_metrics = ['accuracy'] # отслеживаемые метрики

#///////////////////////////////// для тренировки

FIT_batch_size = 800 # размер батчей
FIT_shuffle = True # перемешивать ли данные
FIT_verbose = 1 # выводить ли прогресс обучения в его процессее
FIT_epochs = 100 # количество эпох обучения
FIT_validation_split = 0.15 # процент валидационных данных, отсекаемых из тестовой выборки
FIT_steps_per_epoch = 3000 # кол-во шагов в эпоху (не используется)

In [4]:
# разделить данные на тренировочные и тестовые
def split(X,Y,factor):
    X_train=X[:factor]
    Y_train=Y[:factor]
    X_test=X[factor:]
    Y_test=Y[factor:]
    return X_train,Y_train,X_test,Y_test

In [5]:
# извлечь датасет из файла
def get_df(path, target_name = '2', serarator = ',', col_names = ['1','2']):
    
    file = pd.read_csv(path, sep = serarator, names = col_names)
    dframe = pd.DataFrame(file[target_name])
    return dframe

In [6]:
# убрать англ символы из всех строк датасета
def remove_english(df):
    temp = list()
    for index, row in df.iterrows():
        temp.append("".join([w for w in row[target] if not re.match(r'[A-Z]+', w, re.I)]))
    return temp

In [7]:
temp_names = ["id","name","text","her1","her2","her3",
                  "her4","her5","her6","her7","her8"] # названия колонок для ориг датасета 

# если оригинальный датасет не обрабатывался - обработать и создать новый
if not os.path.exists(path_to_processed_positive_data):
    pos_df = get_df(path_to_positive_data, target, ";", temp_names) # вытащит данные из определенногог столбца с опр сепаратором
    
    pos_text = remove_english(pos_df)
    pos_df = pd.DataFrame(pos_text)
    
    # сохранить как csv (чтобы каждый раз не делать долгую обработку)
    pos_df.to_csv(path_to_processed_positive_data, header = False)
else:
    pos_df = get_df(path_to_processed_positive_data) # вытащит данные из единственного столбца
    pos_text = pos_df['2'].values.tolist()
    
# если оригинальный датасет не обрабатывался - обработать и создать новый    
if not os.path.exists(path_to_processed_negative_data):
    neg_df = get_df(path_to_negative_data, target, ";", temp_names) # вытащит данные из определенногог столбца с опр сепаратором
    
    neg_text = remove_english(neg_df)
    neg_df = pd.DataFrame(neg_text)
    
    # сохранить как csv (чтобы каждый раз не делать долгую обработку)
    neg_df.to_csv(path_to_processed_negative_data, header = False) 
else:
    neg_df = get_df(path_to_processed_negative_data) # вытащит данные из единственного столбца
    neg_text = neg_df['2'].values.tolist()

    
print("pos_text: ", np.array(pos_text[:5]), "\n")
print("neg_text: ", np.array(neg_text[:5]))

pos_text:  ['@_ хоть я и школота, но поверь, у нас то же самое : общество профилирующий предмет типа)'
 'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:'
 ' @: Ну ты идиотка) я испугалась за тебя!!!'
 ' @2912: "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" : ://./62…'
 '@_ Вот что значит страшилка :\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :'] 

neg_text:  ['на работе был полный пиддес :| и так каждое закрытие месяца, я же свихнусь так :'
 'Коллеги сидят рубятся в  , а я из-за долбанной винды не могу :('
 '@_4 как говорят обещаного три года ждут...(('
 'Желаю хорошего полёта и удачной посадки,я буду очень сильно скучать( ://./3'
 'Обновил за каким-то лешим , теперь не работает простоплеер :(']


In [8]:
# создание списка из всех выражений
texts = pos_text + neg_text
count_true = len(pos_text)
count_false = len(neg_text)
total_lines = count_true + count_false
print(
    "pos_text_num: ", count_true, 
    "\nneg_text_num: ", count_false,
    "\nall_text_num: ", total_lines)

pos_text_num:  114911 
neg_text_num:  111923 
all_text_num:  226834


In [9]:
# создание тренировочной/тестовой выборок
X_data = np.array(texts)

Y_data = np.array([1]*count_true + [0]*count_false)
print(X_data.shape, Y_data.shape)

# создание массива индексов чтобы перемешать данные
indeces = np.random.choice(X_data.shape[0], size = X_data.shape[0], replace=False)

print("indeces: ", indeces, "\n")

# перемешивание данных
X_data = X_data[indeces]
Y_data = Y_data[indeces]

# соотношение тренировочной выборки к тестовой
factor = int(.80 * X_data.shape[0])

X_train,Y_train,X_test,Y_test = split(X_data,Y_data,factor)

print("train: ", X_train.shape, Y_train.shape, type(X_train), type(Y_train))
print("test: ", X_test.shape, Y_test.shape, type(X_test), type(Y_test), "\n")

for x in range(5):
    print(X_train[x], Y_train[x])

(226834,) (226834,)
indeces:  [ 48642 208321  32328 ... 198974  69071 119132] 

train:  (181467,) (181467,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
test:  (45367,) (45367,) <class 'numpy.ndarray'> <class 'numpy.ndarray'> 

Планшет шалит. Перемещает картинки из одной папки в другую. У меня фото Уэя в фартучке в папке с котятами. Хотя стойте, все правильно: 1
мои съёмки постоянно куда то переносятся.
я когда нибудь отщелкаю пленку? Мю:( 0
 @_: Ну поймёт только лучшая подруга :) 1
@ @ бизнес с друзьями заводить нельзя :( 0
 @_: Настя подари мне щастья подари мне радость подари любовь :
Азазаза 1


In [10]:
#Sequential model с преобработкой
model = keras.models.load_model("EmotionRecognition_LSTM_NetWork")


In [16]:
# вывести пару результатов работы
for x in range(10,20):    
    a = np.array([X_test[x]])
    
    res = model.predict(a)  
    mark = "Позитивно" if np.mean(res) > 0.5 else "Негативно"
    mark2 = "Позитивно" if Y_test[x] == 1 else "Негативно"
    equal = "==" if mark == mark2 else "  "
    
    print((f"Maybe[{mark}] [%.4f] " % np.mean(res)) , equal , f" Right[{mark2}]\t" , X_test[x])


Maybe[Позитивно] [0.7062]  ==  Right[Позитивно]	 @ никаких колебаний))) все ровно))) на бум бум,значит на бум бум)))
Maybe[Позитивно] [0.8802]      Right[Негативно]	 @ ой, а кстати, как твое творчество там поживает хоть? Очень скучаю за твоими работами :(
Maybe[Позитивно] [0.7511]  ==  Right[Позитивно]	 @2584 @ @ @_ @_ спасибо:3 и тебе добра))
Maybe[Негативно] [0.4417]      Right[Позитивно]	 Жвачка Орбит — укрепляет не только зубы, но и парты в школе) ://./3
Maybe[Позитивно] [0.6695]  ==  Right[Позитивно]	 Добрый день) / Как ваш понедельник? ://./3
Maybe[Негативно] [0.4943]      Right[Позитивно]	 вообщем что хотела сделать придя домой,так и не сделала)
_______________
в этом вся Я:
Maybe[Позитивно] [0.5973]  ==  Right[Позитивно]	 "Это платье на мне плохо сидит" - "Да,на фото как-то кривовато".Да,Тань?)
Maybe[Негативно] [0.1990]  ==  Right[Негативно]	 Ну вот опять начался тот период когда всем насрать на тебя(
Maybe[Негативно] [0.0442]  ==  Right[Негативно]	 ибо мне хреново:( #
Maybe[Не

In [12]:
# тест модели
model.evaluate(X_test, Y_test, batch_size = FIT_batch_size)

57/57 [==============================] - 1s 12ms/step - loss: 0.5689 - accuracy: 0.6978


[0.5689471364021301, 0.6977759003639221]

In [13]:
# подать строку в модель
t = "хохлы люди"

a = list()
a.append(t)
a = np.array(a)

res = model.predict(a)

mark = "Положительно" if np.mean(res) > 0.5 else "отрицательно"

print(f"mark [{mark}]")
print("res: ", res)
print("res[0,0]: ", res[0,0])

mark [отрицательно]
res:  [[0.41758302]]
res[0,0]:  0.41758302


In [14]:
# подать строку в модель
t = "это просто ужасно плохо противно не хорошо"

a = list()
a.append(t)
a = np.array(a)

res = model.predict(a)

mark = "Положительно" if np.mean(res) > 0.5 else "отрицательно"

print(f"mark [{mark}]")
print("res: ", res)
print("res[0,0]: ", res[0,0])

mark [отрицательно]
res:  [[0.0390704]]
res[0,0]:  0.039070398
